In [2]:
import datetime
import decimal
import pydantic
import enum
from typing import *
import warnings
import QuantLib as ql
import pandas as pd
import numpy as np

import finsec as fs

In [3]:
# fs.fixed_income_objs.Period(period='1y').model_dump()

## autoeload info:
%load_ext autoreload
%autoreload 2

In [4]:
# BusinessDayConvention.following.to_ql()
# BusinessDayConvention.from_ql(ql.Following)

In [5]:
import json
import functools
from operator import add, sub, mul

T = TypeVar('T')
ListOrT = Union[List[T], T]



In [8]:
acc1 = fs.AccrualInfo(
    start=datetime.date(2025,1,1),
    # end=datetime.date(2025,11,5),
    end='3m',
    period='1m',
    dc=fs.DayCount.Thirty360,
    front_stub_not_back=False,
)
# acc1.as_ql().dates()
print(len(acc1))
acc1.schedule().to_df()

3


,start,end,frac
0,2025-01-01,2025-02-01,0.083333
1,2025-02-01,2025-03-01,0.083333
2,2025-03-01,2025-04-01,0.083333


In [9]:
print(json.dumps(json.loads(acc1.model_dump_json()), indent=4))
fs.AccrualInfo.model_validate_json(acc1.model_dump_json())

{
    "start": "2025-01-01",
    "end": {
        "period": "3M"
    },
    "dc": "30/360",
    "freq": null,
    "cal_accrual": "null",
    "cal_pay": "null",
    "period": {
        "period": "1M"
    },
    "bdc": "F",
    "front_stub_not_back": false,
    "eom": false
}


AccrualInfo(start=datetime.date(2025, 1, 1), end=Period("3M"), dc=<DayCount.Thirty360: '30/360'>, freq=None, cal_accrual=<Calendar.NULL: 'null'>, cal_pay=<Calendar.NULL: 'null'>, period=Period("1M"), bdc=<BusinessDayConvention.following: 'F'>, front_stub_not_back=False, eom=False)

In [10]:
expr1 = fs.FixedRate(rate=decimal.Decimal('0.05')) * (decimal.Decimal('1')/10) + 10

expr1.get_fixing(None), expr1.is_constant

(Decimal('10.005'), True)

In [11]:
dc = ql.Actual360()

# for x in dir(dc):
#     print(x)

In [12]:
expr1.model_dump()

{'components': [{}, {}], 'operator': <ExprOperator.ADD: 'ADD'>}

In [13]:
ntnl = 1_000_000
fixleg1 = fs.Leg(
    notional=ntnl,
    cpn=fs.FixedRate(rate=decimal.Decimal('0.01')),
    acc=fs.AccrualInfo(
        start=datetime.date(2025,1,1),
        end=datetime.date(2026,1,1),
        dc=fs.DayCount.Thirty360,
        freq=12,
        bdc=fs.BusinessDayConvention.unadjusted,
    )
)
# print(fixleg1)

bnd1 = fs.Bond(
    notional=ntnl,
    leg=fixleg1,
    # settle: datetime.date
    settle_days=1
)
print(bnd1)
ql_bnd1 = bnd1.as_quantlib()

notional=Decimal('1000000') leg=Leg(notional=Decimal('1000000'), cpn=FixedRate(rate=Decimal('0.01'), is_constant=True, is_float=False), acc=AccrualInfo(start=datetime.date(2025, 1, 1), end=datetime.date(2026, 1, 1), dc=<DayCount.Thirty360: '30/360'>, freq=Decimal('12'), cal_accrual=<Calendar.NULL: 'null'>, cal_pay=<Calendar.NULL: 'null'>, period=None, bdc=<BusinessDayConvention.unadjusted: 'U'>, front_stub_not_back=True, eom=False), pay_delay=None) settle_days=1 credit_index=None face=100 settle=None redemption=None


In [15]:
bnd1.cashflows_df().style.format({'amount':'{0:,.2f}'})

,start,end,frac,date,amount,has_occurred,rate
0,2025-01-01,2025-02-01,0.083333,2025-02-01,833.33,True,0.010000
1,2025-02-01,2025-03-01,0.083333,2025-03-01,833.33,True,0.010000
2,2025-03-01,2025-04-01,0.083333,2025-04-01,833.33,True,0.010000
3,2025-04-01,2025-05-01,0.083333,2025-05-01,833.33,True,0.010000
4,2025-05-01,2025-06-01,0.083333,2025-06-01,833.33,True,0.010000
5,2025-06-01,2025-07-01,0.083333,2025-07-01,833.33,True,0.010000
6,2025-07-01,2025-08-01,0.083333,2025-08-01,833.33,True,0.010000
7,2025-08-01,2025-09-01,0.083333,2025-09-01,833.33,True,0.010000
8,2025-09-01,2025-10-01,0.083333,2025-10-01,833.33,False,0.010000
9,2025-10-01,2025-11-01,0.083333,2025-11-01,833.33,False,0.010000


In [16]:
bnd1.model_dump()

/config/workspace/venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `decimal` - serialized value may not be as expected [input_value=100, input_type=int])
  return self.__pydantic_serializer__.to_python(


{'notional': Decimal('1000000'),
 'leg': {'notional': Decimal('1000000'),
  'cpn': {'rate': Decimal('0.01'), 'is_constant': True, 'is_float': False},
  'acc': {'start': datetime.date(2025, 1, 1),
   'end': datetime.date(2026, 1, 1),
   'dc': <DayCount.Thirty360: '30/360'>,
   'freq': Decimal('12'),
   'cal_accrual': <Calendar.NULL: 'null'>,
   'cal_pay': <Calendar.NULL: 'null'>,
   'period': None,
   'bdc': <BusinessDayConvention.unadjusted: 'U'>,
   'front_stub_not_back': True,
   'eom': False},
  'pay_delay': None},
 'settle_days': 1,
 'credit_index': None,
 'face': 100,
 'settle': None,
 'redemption': None}

In [18]:
bnd2 = fs.Bond.model_validate_json(bnd1.model_dump_json())
bnd2.model_dump()

/config/workspace/venv/lib/python3.12/site-packages/pydantic/main.py:519: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `decimal` - serialized value may not be as expected [input_value=100, input_type=int])
  return self.__pydantic_serializer__.to_json(


{'notional': Decimal('1000000'),
 'leg': {'notional': Decimal('1000000'),
  'cpn': {'rate': Decimal('0.01'), 'is_constant': True, 'is_float': False},
  'acc': {'start': datetime.date(2025, 1, 1),
   'end': datetime.date(2026, 1, 1),
   'dc': <DayCount.Thirty360: '30/360'>,
   'freq': Decimal('12'),
   'cal_accrual': <Calendar.NULL: 'null'>,
   'cal_pay': <Calendar.NULL: 'null'>,
   'period': None,
   'bdc': <BusinessDayConvention.unadjusted: 'U'>,
   'front_stub_not_back': True,
   'eom': False},
  'pay_delay': None},
 'settle_days': 1,
 'credit_index': None,
 'face': Decimal('100'),
 'settle': None,
 'redemption': None}

In [19]:
bnd1.model_dump()

/config/workspace/venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `decimal` - serialized value may not be as expected [input_value=100, input_type=int])
  return self.__pydantic_serializer__.to_python(


{'notional': Decimal('1000000'),
 'leg': {'notional': Decimal('1000000'),
  'cpn': {'rate': Decimal('0.01'), 'is_constant': True, 'is_float': False},
  'acc': {'start': datetime.date(2025, 1, 1),
   'end': datetime.date(2026, 1, 1),
   'dc': <DayCount.Thirty360: '30/360'>,
   'freq': Decimal('12'),
   'cal_accrual': <Calendar.NULL: 'null'>,
   'cal_pay': <Calendar.NULL: 'null'>,
   'period': None,
   'bdc': <BusinessDayConvention.unadjusted: 'U'>,
   'front_stub_not_back': True,
   'eom': False},
  'pay_delay': None},
 'settle_days': 1,
 'credit_index': None,
 'face': 100,
 'settle': None,
 'redemption': None}

In [20]:
swp_ntnl = 1_000_000
swp_start = datetime.date(2025,1,1)
swp_end = '5y'
swp_index = 'SOFR'
swp = fs.Swap.make_ois(
    start=datetime.date(2025, 1, 1),
    end='5y',
    rate=3.5 / 100,
    dc_fix=fs.DayCount.Actual360,
    dc_float=fs.DayCount.Actual360,
    freq_fix=1,
    freq_float=1,
    index='SOFR',
    cal_pay=fs.Calendar.US_SOFR,
    notional=1_000_000,
    pay_delay=fs.Period(period='2d'),
)
display(swp.model_dump())

{'legs': ({'notional': Decimal('1000000'),
   'cpn': {'rate': Decimal('0.035'), 'is_constant': True, 'is_float': False},
   'acc': {'start': datetime.date(2025, 1, 1),
    'end': {'period': '5Y'},
    'dc': <DayCount.Actual360: 'act/360'>,
    'freq': Decimal('1'),
    'cal_accrual': <Calendar.NULL: 'null'>,
    'cal_pay': <Calendar.US_SOFR: 'us/sofr'>,
    'period': None,
    'bdc': <BusinessDayConvention.following: 'F'>,
    'front_stub_not_back': True,
    'eom': False},
   'pay_delay': None},
  {'notional': Decimal('-1000000'),
   'cpn': {'index': 'SOFR',
    'type_': {'type_': 'overnight',
     'pay_delay': {'period': '2D'},
     'compounded_not_averaged': True},
    'is_constant': False,
    'is_float': True},
   'acc': {'start': datetime.date(2025, 1, 1),
    'end': {'period': '5Y'},
    'dc': <DayCount.Actual360: 'act/360'>,
    'freq': Decimal('1'),
    'cal_accrual': <Calendar.NULL: 'null'>,
    'cal_pay': <Calendar.US_SOFR: 'us/sofr'>,
    'period': None,
    'bdc': <Busines

In [21]:
swp.model_dump()

{'legs': ({'notional': Decimal('1000000'),
   'cpn': {'rate': Decimal('0.035'), 'is_constant': True, 'is_float': False},
   'acc': {'start': datetime.date(2025, 1, 1),
    'end': {'period': '5Y'},
    'dc': <DayCount.Actual360: 'act/360'>,
    'freq': Decimal('1'),
    'cal_accrual': <Calendar.NULL: 'null'>,
    'cal_pay': <Calendar.US_SOFR: 'us/sofr'>,
    'period': None,
    'bdc': <BusinessDayConvention.following: 'F'>,
    'front_stub_not_back': True,
    'eom': False},
   'pay_delay': None},
  {'notional': Decimal('-1000000'),
   'cpn': {'index': 'SOFR',
    'type_': {'type_': 'overnight',
     'pay_delay': {'period': '2D'},
     'compounded_not_averaged': True},
    'is_constant': False,
    'is_float': True},
   'acc': {'start': datetime.date(2025, 1, 1),
    'end': {'period': '5Y'},
    'dc': <DayCount.Actual360: 'act/360'>,
    'freq': Decimal('1'),
    'cal_accrual': <Calendar.NULL: 'null'>,
    'cal_pay': <Calendar.US_SOFR: 'us/sofr'>,
    'period': None,
    'bdc': <Busines

## bootstrap a treasury curve

In [22]:
class SingleCurveConfig(pydantic.BaseModel):
    name: Hashable
    date: datetime.date
    settle_days: int
    dc: fs.DayCount
    cal: fs.Calendar

    # parallel_bump:fs.Quote = pydantic.Field(default_factory=fs.Quote)
    resets: Dict[datetime.date, decimal.Decimal] = pydantic.Field(default_factory=dict)
    helpers: Dict[str, Tuple[fs.Quote, Any]] = pydantic.Field(default_factory=dict)

    ql_curve_raw: ql.YieldTermStructure|None=None
    handle: ql.RelinkableYieldTermStructureHandle=pydantic.Field(default_factory=ql.RelinkableYieldTermStructureHandle)
    index: ql.InterestRateIndex|None=None

    class Config:
        arbitrary_types_allowed = True
    
    def benchmarks(self)->Dict[str, fs.Quote]:
        return { n:q[0] for n,q in self.helpers.items() }

    def add_helper(self, name:str, quote:fs.Quote, helper:Any):
        self.helpers[name] = (quote, helper)
    
    def add_reset(self, dt:datetime.date, rate:decimal.Decimal):
        self.resets[dt] = rate
        if self.index is not None:
            self.index.addFixing( ql.Date.from_date(dt), float(rate))
    
    def build_quantlib_index(self):
        usd = ql.USDCurrency()
        self.index = ql.OvernightIndex(
            self.name,
            self.settle_days,
            usd,
            self.cal.as_ql(),
            self.dc.as_ql,
            self.handle
        )
        self.index.addFixings(
            [ql.Date.from_date(x) for x in self.resets.keys()],
            [float(x) for x in self.resets.values()],
        )

    def build_quantlib_curve(self):
        instruments = [x[1] for x in self.helpers.values()]
        self.ql_curve_raw = ql.PiecewiseFlatForward(
            ql.Date.from_date(self.date),
            instruments,
            self.dc.as_ql,
        )
        self.handle.linkTo(self.ql_curve_raw)
    
    # def plot_forwards()

# class CurveInterp(pydantic.BaseModel):
#     order: List[Literal[0,1,2]]
#     joint_date:
    



In [23]:
tsy_curve = SingleCurveConfig(
    name='USGOVT',
    date=datetime.date(2025, 1, 1),
    settle_days=1,
    dc=fs.DayCount.ActualActual,
    cal=fs.Calendar.US_GovernmentBond,
)

In [24]:
bond_tenors = ['1y','2y','5y','10y']
bonds = [
    fs.Bond(
        credit_index='USGOVT',
        notional=ntnl,
        leg=fs.Leg(
            notional=100,
            cpn=fs.FixedRate(rate=5.0 / 100),
            acc=fs.AccrualInfo(
                start=datetime.date(2025,1,1),
                end=ten,
                dc=fs.DayCount.ActualActual,
                freq=2,
                bdc=fs.BusinessDayConvention.modified_following,
            )
        ),
        settle_days=1
    )
    for ten in bond_tenors
]

# bond_prices = [100] * len(bond_tenors)
bond_prices = [
    100,
    99,
    98,
    97,
]
bond_helpers = [bnd.as_quantlib_helper() for bnd in bonds]
for px, q in zip(bond_prices, bond_helpers):
    q[0].setValue(px)

In [26]:
for ten, qh in zip(bond_tenors, bond_helpers):
    q,h = qh
    tsy_curve.add_helper( ten, q, h)

In [27]:
tsy_curve.build_quantlib_curve()
tsy_curve.ql_curve_raw.enableExtrapolation()

In [28]:
# tsy_curve.plot(datetime.date(2025,1,1), datetime.date(2035,1,1))

In [29]:
# swp_crv0_ql = ql.PiecewiseFlatForward(
# )

# help(ql.PiecewiseFlatForward)
# PiecewiseFlatForward self
#     Date referenceDate
#     RateHelperVector instruments
#     DayCounter dayCounter
#     QuoteHandleVector jumps=std::vector< Handle< Quote > >()
#     DateVector jumpDates=std::vector< Date >()
#     BackwardFlat i=BackwardFlat()
#     IterativeBootstrap b=_IterativeBootstrap()

for x in dir(ql):
    if 'piecewise' in x.lower():
        print(x)

PiecewiseConstantCorrelation
PiecewiseConstantParameter
PiecewiseConvexMonotoneForward
PiecewiseConvexMonotoneZero
PiecewiseCubicZero
PiecewiseFlatForward
PiecewiseFlatHazardRate
PiecewiseForwardSpreadedTermStructure
PiecewiseKrugerLogDiscount
PiecewiseKrugerZero
PiecewiseLinearForward
PiecewiseLinearForwardSpreadedTermStructure
PiecewiseLinearZero
PiecewiseLogCubicDiscount
PiecewiseLogLinearDiscount
PiecewiseLogMixedLinearCubicDiscount
PiecewiseLogParabolicCubicDiscount
PiecewiseMonotonicLogParabolicCubicDiscount
PiecewiseMonotonicParabolicCubicZero
PiecewiseNaturalCubicZero
PiecewiseNaturalLogCubicDiscount
PiecewiseParabolicCubicZero
PiecewiseSplineCubicDiscount
PiecewiseTimeDependentHestonModel
PiecewiseYoYInflation
PiecewiseZeroInflation
PiecewiseZeroSpreadedTermStructure


In [ ]:
# help(ql)